### Import needed libraries

In [ ]:
# library provided by EPOCH for reading .sdf output files into Python
import sdf_helper as sh 

# python plotting library similar to MATLAB(TM)
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# signal processing, needed for convolution
from scipy import signal

# Python array manipulation
import numpy as np

# various math functions 
import math as m

In [ ]:
# show plots in the notebook
%matplotlib inline

### Define useful functions

In [ ]:
from collections import OrderedDict

In [ ]:
def get_sdf_files(path):
    r"""Given a ``path``, it returns an ordered dictionary containing all the .sdf files in that path, 
    in the form {#### : /../../####.sdf}"""
    
    sdfs = glob.glob(os.path.join(path, '*.sdf')) 
    sdf_dict = {int(sdf.split('.')[0][-4:]):sdf for sdf in sdfs}
    sdf_ord_dict = OrderedDict(sorted(sdf_dict.items(), key=lambda t: t[1])) 
    
    return sdf_ord_dict

In [ ]:
def center_linspace(space):
    r"""
    
    Parameters
    ----------
    space : ndarray
        A one-dimensional array, of linearly spaced numbers. 
    
    Returns
    -------
    out : ndarray
        Also an array of the same spacing, this time centered around 0.
    """
    
    assert np.ndim(space) == 1
    size = space.size
    first = space[0]
    last = space[-1]
    length = last - first
    end = length/2
    
    _, step = np.linspace(first, last, size, retstep=True)
    
    out, out_step = np.linspace(-end, end, size, retstep=True) 
    assert m.isclose(out_step, step)
    return out

In [ ]:
def list_sdf_variables(data):
    r"""Lists all the quantities from the .sdf file.
    
    Parameters
    ----------
    data : ``sdf.Blocklist``
        The results of calling sdf.read on an .sdf file.
    """
    dct = data.__dict__
    for key in sorted(dct):
        try:
            val = dct[key]
            print('{} {} {}'.format(key, type(val),
                  np.array2string(np.array(val.dims), separator=', ')))
        except:
            pass

In [ ]:
def colorbar(mappable):
    r"""Constructs a scaled colorbar for a given plot.
    
    Parameters
    ----------
    mappable : The Image, ContourSet, etc. to which the colorbar applies.
    """
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    return fig.colorbar(mappable, cax=cax)

In [ ]:
def gauss_kern_nd(n, sizes):
    r"""Constructs a Gaussian kernel in any number of dimensions.
    
    Parameters
    ----------
    n : int
        The number of dimensions. Must be 1 or above.
    sizes : list
        The sizes along the various dimensions, ie. in 3D this would be [size_x, size_y, size_x]. The length of
        this list should be either 1 or n. If it only contains one element, eg. [size], it is assumed that
        size_x = size_y = size_x = size.
    
    Returns
    -------
    g : ndarray
        The kernel, with dimensions (in 3d) (2*size_x+1, 2*size_y+1, 2*size_z+1).
    """
    assert n > 0, 'at least 1d required'
    no_sizes = len(sizes)
    assert no_sizes == 1 or no_sizes == n, 'either give one size or all of them'
    
    for i in range(n - no_sizes):
        sizes.append(sizes[0])
        
    slices = tuple(slice(-size,size+1,None) for size in sizes)
    
    XXX = np.mgrid[slices]
    
    g = np.ones(XXX.shape[1:], dtype=np.float64)
    
    for X, size in zip(XXX, sizes):
        g = g * np.exp(-X**2/size)
        
    return g / g.sum()

In [ ]:
def smooth(data, n, sizes):
    r"""Smoothens the input by performing a convolution with a Gaussian kernel.
    
    Parameters
    ----------
    data : ndarray
        Input data, with ``n`` dimensions.
    n : int
        The number of dimensions. Must be 1 or above.
    sizes : list
        The sizes along the various dimensions, ie. in 3D this would be [size_x, size_y, size_x]. The length of
        this list should be either 1 or n. If it only contains one element, eg. [size], it is assumed that
        size_x = size_y = size_x = size.
        
    Returns
    -------
    out : ndarray
        The smoothed input, of shape ```data.shape - 2 * sizes```.
    """
    
    g = gauss_kern_nd(n, sizes)
    out = signal.convolve(data, g, mode='valid')
    return(out)

bash console commands can be run inside the notebook, eg.

The following code will download the data needed for this notebook automatically using `curl`. It may take some time (the archive is 1.2 GB), so please wait when the kernel is busy. You will need to set `download_datasets` to `True` before using it.

In [ ]:
download_datasets = False
if download_datasets:
    !curl -sSO https://ndownloader.figshare.com/articles/5545165/versions/1
    print ("Downloaded the EPOCH data from figshare.")
    !unzip 1 
    
    print ("All done!")

In [ ]:
# these are the .sdf files used in the notebook. they must be present in the same folder when the notebook is run
!ls -lsa *.sdf

In [ ]:
# these are the corresponding EPOCH input decks
!ls -lsa *.deck

# 2D case

In [ ]:
!cat 2dinput.deck

Now we analythe the results from a simple 2D simulation. We use the higher-level library [`holoviews`](http://holoviews.org) for interactive plotting.

In [ ]:
fname = '2d.sdf'
data_2d = sh.getdata(fname);

In [ ]:
sh.list_variables(data_2d)

In [ ]:
grid = data_2d.Grid_Grid_mid
nele = data_2d.Derived_Number_Density_ele
ex = data_2d.Electric_Field_Ex
ey = data_2d.Electric_Field_Ey

In [ ]:
sh.plot_auto(nele)

In [ ]:
(x, y) = grid.data
# convert to micrometers
x = x*1e+6
y = y*1e+6

In [ ]:
# get boundaries of simulation box
x_min = np.min(x); x_max = np.max(x)
y_min = np.min(y); y_max = np.max(y)

In [ ]:
# note we transpose the raw data 
rho = nele.data.T

# we convolute with a Gaussian kernel of size (30, 30) to reduce the noise
kern_size = 30
rho_smooth = smooth(nele.data.T, 2, [kern_size])

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=[14,4])

imgs = []
for ax, r in zip(axes, (rho, rho_smooth)):
    img = ax.imshow(r, origin='lower', interpolation='none',
                    extent=np.array([x_min, x_max, y_min, y_max]),
                    aspect='auto')
    imgs.append(img)

axes[0].set_title('raw data')
axes[1].set_title('after convolution with Gaussian kernel')

for ax in axes:
    for axis, label in zip([ax.xaxis, ax.yaxis], nele.grid_mid.labels):
        axis.set(label_text=label + r' $(\mu m)$')

# add colorbars to the plots
cbars = [colorbar(img) for img in imgs]

for cbar in cbars:
    cbar.set_label(nele.name + r' $(' + nele.units + r')$')

plt.tight_layout(h_pad=1)

In [ ]:
# electric field, x and y components
efx = ex.data.T
efy = ey.data.T

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=[14,4])

imgs = []
for ax, ef in zip(axes, (efx, efy)):
    img = ax.imshow(ef, origin='lower', interpolation='none',
                    extent=np.array([x_min, x_max, y_min, y_max]),
                    aspect='auto')
    imgs.append(img)

for ax, ef in zip(axes, (ex,ey)):
    for axis, label in zip([ax.xaxis, ax.yaxis], ef.grid_mid.labels):
        axis.set(label_text=label + r' $(\mu m)$')

# add colorbars to the plots
cbars = [colorbar(img) for img in imgs]

for cbar, ef in zip(cbars, (ex, ey)):
    cbar.set_label(ef.name + r' $(' + ef.units + r')$')

plt.tight_layout(h_pad=1)

In [ ]:
# integrate out the x dependence
e_sum_x = np.sum(np.abs(efy), axis=1)

In [ ]:
# we get a Gaussian
fig, ax = plt.subplots()
ax.plot(y, e_sum_x)
ax.set_xlabel(ey.grid_mid.labels[1] + r' $(\mu m)$', labelpad=0)
ax.set_ylabel(r'$\sum_{x} |E_y(x,y)|$ [V/m]');

In [ ]:
# interactive visualization library
import holoviews as hv
hv.extension('matplotlib')

In [ ]:
# set global plotting and style options for holoviews
%opts Image (cmap='viridis') [colorbar=True] Bounds (color='red') HLine (color='red') VLine (color='red')

We want to look at the electron density. We first define its dimensions, along with the corresponding units.

In [ ]:
xdim = hv.Dimension('x', label='x', unit=r'$\mu$'+'m')
ydim = hv.Dimension('y', label='y', unit=r'$\mu$'+'m')
zdim = hv.Dimension('z', label=nele.name, unit=r'$'+nele.units+r'$')

In [ ]:
# this are the ranges of the axes after the convolution
k = kern_size
xrange = x[k:-k]
yrange = y[k:-k]

In [ ]:
# we load the smoothed electron density into an Image object
img = hv.Image((xrange, yrange, rho_smooth), datatype=['grid'], 
                 kdims=[xdim, ydim], vdims=[zdim])

In [ ]:
# we want to focus on a particular region of the density, defined by these coordinates
x1 = xrange[60]; x2 = xrange[-500]
y1 = -20; y2 = 20

In [ ]:
# we plot the full image and the region of interest side by side

In [ ]:
%%output size=150
img_box = img[x1:x2, y1:y2]
box = hv.Bounds((x1,y1,x2,y2))
img*box + img_box

Now we can define a slice along the `y` axis and interactively move it. 

In [ ]:
x_slice = {x : img_box * hv.VLine(x=x) + img_box.sample(x=x) for x in np.linspace(x1, x2, 50)}

In [ ]:
hv.HoloMap(x_slice, kdims=['x']).collate()

And of course, we can do the same along `x`.

In [ ]:
y_slice = {y : img_box * hv.HLine(y=y) + img_box.sample(y=y) for y in np.linspace(y1, y2, 50)}

In [ ]:
hv.HoloMap(y_slice, kdims=['y']).collate()

And we can plot the region of interest, together with the two slices, in one line. Notice `holoviews` automatically knows how to label the plots.

In [ ]:
%%output size=150
img_box * hv.HLine(y=0.) * hv.VLine(x=787) + img_box.sample(y=0.) + img_box.sample(x=787)